# Equational

```python
from logex import equational as eq
```

Small sub-module for equational reasoning

## 1. Terms

The main building block of equational reasoning is a Term.

```python
eq.Term(term: str)
```

IMPORTANT: some functions may also work with str

By conventions, in term:

- fuctions are written with small letters
- variables are written with capital letters
- constants can be written like function with empty input or without any input (c or c())

In [1]:
from logex import equational as eq

tau = eq.Term('g(f (f (Y )), X )')
print(tau.info())

Term: g(f(f(Y)),X)
Originals: g(f (f (Y )), X )
Ground term? False
Functions: {'f', 'g'}
with arities: {'g': 2, 'f': 1}
Variables: {'Y', 'X'}
Constants: set()
Sub-terms: {'g(f(f(Y)),X)', 'f(Y)', 'X', 'f(f(Y))', 'Y'}



Sometimes is it important that 2 terms do not have common variables:
```python
disjoin_variables(to_keep: str | Term, 
                  to_replace: str | Term) -> tuple[Term, Term]:
```

In [2]:
tau = eq.Term('g(f (f (Y )), X )')
tau_ = eq.Term('g(X, f(Y ))')

tau, tau_renamed = eq.disjoin_variables(to_keep=tau, to_replace=tau_)
print(tau)
print(tau_renamed)

g(f(f(Y)),X)
g(G,f(V))


Two terms can be matched by finding a substitution, such that it converts one term to another term (or to show that there is no such substitution):

```python
eq.match_terms(original_term: str | Term, 
               target_term: str | Term) -> dict
```

In [3]:
tau1 = eq.Term('f (X, g(Y, Z)) ')
tau2 = eq.Term("f (h(Z'), g (X', Y'))")

eq.match_terms(tau1, tau2);

###################### TERM MATCHING #######################
Original term: f(X,g(Y,Z))
Target term: f(h(Z'),g(X',Y'))
Mismatch found:
	f(X,g(Y,Z))
	f(h(Z'),g(X',Y'))
	  ^

Substitution added: X -> h(Z')

Mismatch found:
	f(h(Z'),g(Y,Z))
	f(h(Z'),g(X',Y'))
	          ^

Substitution added: Y -> X'

Mismatch found:
	f(h(Z'),g(X',Z))
	f(h(Z'),g(X',Y'))
	             ^

Substitution added: Z -> Y'

Result: {'Y': "X'", 'X': "h(Z')", 'Z': "Y'"}
############################################################



## 2. Generalization / Unification

There are 2 values of interest:

- LGG (least general generalization)
- MGU (most general unifier)

```python
least_general_generalization(term_1: str | Term, 
                             term_2: str | Term) -> Term, dict, dict
```

Returns LGG and 2 substitutions that match LGG to the original terms

In [4]:
tau1 = eq.Term('f (X, g(Z))')
tau2 = eq.Term("f (h(X), Y )")

eq.least_general_generalization(tau1, tau2);

########################### LGG ############################
Term 1: f(X,g(Z))
Term 2: f(h(X),Y)
Mismatch found:
	f(X,g(Z))
	f(h(X),Y)
	  ^

Substitution added to sigma_1: V -> X
Substitution added to sigma_2: V -> h(X)
Back substitution is performed

Mismatch found:
	f(V,g(Z))
	f(V,Y)
	    ^

Substitution added to sigma_1: G -> g(Z)
Substitution added to sigma_2: G -> Y
Back substitution is performed

Result: f(V,G)
sigma_1: {'V': 'X', 'G': 'g(Z)'}
sigma_2: {'V': 'h(X)', 'G': 'Y'}
############################################################



```python
most_general_unifier(term_1: str | Term, 
                     term_2: str | Term) -> dict | bool
```

Returns MGU if terms are unifiabe, else False

In [5]:
tau1 = eq.Term('g(X,f(f(Y)))')
tau2 = eq.Term("g(g(Z,Y),f(Z))")

eq.most_general_unifier(tau1, tau2);

########################### MGU ############################
Term 1: g(X,f(f(Y)))
Term 2: g(g(Z,Y),f(Z))
Mismatch found:
	g(X,f(f(Y)))
	g(g(Z,Y),f(Z))
	  ^

New substitution is added: X -> g(Z,Y)
Mismatch found:
	g(g(Z,Y),f(f(Y)))
	g(g(Z,Y),f(Z))
	           ^

New substitution is added: Z -> f(Y)
Result: {'X': 'g(f(Y),Y)', 'Z': 'f(Y)'}
Final unification: g(g(f(Y),Y),f(f(Y)))
############################################################



## 3.Reduction Systems

Term Order is a Callable object that takes 2 terms and returns TRUE if the first term is LESS that the second term, FALSE - if the second term is less than the first, and None if they are incomparable.

Rules for comparison might be defined in different ways. Here is a built-in implementation for defult term ordering:

In [6]:
order = eq.DefaultTermOrder()

print(order('X', 'f(X)'))
print(order('f(X)', 'g(X)'))
print(order('f(X)', 'g(X, Y)'))
print(order('f(X, Y)', 'g(X)'))
print(order('f(X, g(X))', 'f(X)'))

None
True
True
False
False


### Reduction
```python
reduction_of_term(reduction_system: Collection, 
                  term: str) -> Term, list, str
```

Tries to perform reduction with respect to the reduction system of the given term

- reduction system: list of tuples, each tuple consists of left part and right part of the reduction rule


In [7]:
R = {
    ('square(X)', 'times(X, X)'),
    ('times(plus(X, Y ), Z )', 'plus(times(X, Z), times(Y, Z))'),
    ('times(X, plus(Y, Z))', 'plus(times(X, Y ), times(X, Z))')
    }
term = 'square(plus(X, one()))'

eq.reduction_of_term(R, term);

######################## REDUCTION #########################
Term: square(plus(X,one))
Reduction system: 
square(X) -> times(X,X)
times(X,plus(Y,Z)) -> plus(times(X,Y),times(X,Z))
times(plus(X,Y),Z) -> plus(times(X,Z),times(Y,Z))

Step 1, by using rule: [ square(X) -> times(X,X) ]:
	square(plus(X,one)) ==> times(plus(V,one),plus(V,one))

Step 2, by using rule: [ times(X,plus(Y,Z)) -> plus(times(X,Y),times(X,Z)) ]:
	times(plus(V,one),plus(V,one)) ==> plus(times(plus(V,one),V),times(plus(V,one),one))

Step 3, by using rule: [ times(plus(X,Y),Z) -> plus(times(X,Z),times(Y,Z)) ]:
	plus(times(plus(V,one),V),times(plus(V,one),one)) ==> plus(plus(times(V,V),times(one,V)),times(plus(V,one),one))

Step 4, by using rule: [ times(plus(X,Y),Z) -> plus(times(X,Z),times(Y,Z)) ]:
	plus(plus(times(V,V),times(one,V)),times(plus(V,one),one)) ==> plus(plus(times(V,V),times(one,V)),plus(times(V,one),times(one,one)))

Result: plus(plus(times(V,V),times(one,V)),plus(times(V,one),times(one,one)))
###########

### Critical Pairs
```python
critical_pairs(reduction_system: list) -> set[tuple]
```

Returns set of critical pairs for a given reduction system

In [8]:
reduction_set = [("f(g(X))", "f(X)"), 
                 ("g(f(Y))", "f(Y)"), 
                 ("h(g(Z))", "f(Z)")]

eq.critical_pairs(reduction_set);

################# CRITICAL PAIRS ALGORITHM #################
Reduction system: 
f(g(X)) -> f(X)
g(f(Y)) -> f(Y)
h(g(Z)) -> f(Z)

Initialising Critical pairs to empty set: set()
1) Working with pair of rules:
			Rule 1: [ f(g(X)) -> f(X) ]
			Rule 1: [ f(g(X)) -> f(X) ]

	λ1 [ f(g(X)) ] and λ2 [ f(g(X)) ] have 1 common variables: {'X'}
	substitution σ = {V <- X}
	New rule 2: [ f(g(V)) -> f(V) ]

	Iterating over non-variable subterms (s) of λ1 [ f(g(X)) ]:

		1.1) Working with a subterm s [ f(g(X)) ]:
			Most general unifier of s and λ2: μ = {'X': 'V'}
			 as μ = renaming => nothing happens


		1.2) Working with a subterm s [ g(X) ]:
			Most general unifier of s and λ2: μ = ⊥
			 as μ = ⊥ => nothing happens

2) Working with pair of rules:
			Rule 1: [ f(g(X)) -> f(X) ]
			Rule 2: [ g(f(Y)) -> f(Y) ]

	λ1 [ f(g(X)) ] and λ2 [ g(f(Y)) ] have 0 common variables: set()
	substitution σ = {}

	Iterating over non-variable subterms (s) of λ1 [ f(g(X)) ]:

		2.1) Working with a subterm s [ f(g(X)

### Knuth-Bendix Completion

```python
knuth_bendix(reduction_system: list, 
             term_order: Callable = DefaultTermOrder()) -> list[tuple]
```

Returns a complete reduction system (with deterministic reduction pipeline)

In [9]:
reduction_set = [("f(g(X))", "f(X)"), 
                 ("g(f(Y))", "f(Y)"), 
                 ("h(g(Z))", "f(Z)")]

eq.knuth_bendix(reduction_set);

####################### KNUTH-BENDIX #######################
Reduction system (STEP 0): 
f(g(X)) -> f(X)
g(f(Y)) -> f(Y)
h(g(Z)) -> f(Z)


Critical pairs were recomputed
Critical pairs left to check (2):
g(f(A)) -> f(g(A))
h(f(A)) -> f(f(A))

Working with critical pair: g(f(A)) -> f(g(A))
Obtained: ρ = f(A) and ρ' = f(A)
Equal results obtained - no new rules deduced.


Critical pairs were recomputed
Critical pairs left to check (1):
h(f(A)) -> f(f(A))

Working with critical pair: h(f(A)) -> f(f(A))
Obtained: ρ = h(f(A)) and ρ' = f(f(A))
Rule h(f(A)) -> f(f(A)) is appended to reduction rules

Critical pairs were recomputed
Critical pairs left to check (1):
h(f(B)) -> f(f(g(B)))

Working with critical pair: h(f(B)) -> f(f(g(B)))
Obtained: ρ = f(f(B)) and ρ' = f(f(B))
Equal results obtained - no new rules deduced.

Reduction system (FINAL): 
f(g(X)) -> f(X)
g(f(Y)) -> f(Y)
h(g(Z)) -> f(Z)
h(f(A)) -> f(f(A))
############################################################
